In [1]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-07-27 10:44:38


# Advanced Installation

This document details installation philosophies and approaches for maintaining reproducible packages.

## Philosphophy on Reproducibility

**Problem**:  The same package can behave differently based on:

1. the dependency environment and versions
1. third-party updates to dependencies
1. operating system (OS)

The first and second items are prevented with pinned dependencies, one method for [repeatable packages](https://pip.pypa.io/en/stable/user_guide/#ensuring-repeatability).  The third item is prevented through continuous integration, specifically with [Travis](https://travis-ci.org/) and [Appveyor](https://www.appveyor.com/) (for Linux and Windows systems respectively).  We will discuss proposals for the first two items only.  

**Proposed Solution**: We take "snapshots" of our build environments before each release cycle.

- *Start*: update then freeze the conda enviroment in `environment.yaml` files
- *During*: update dependencies as needed `requirements.txt` and `dev_requirements.txt`

## Reproducibility in LamAna

Reproducible enviroments are important when dependencies conflict with the package due to bugged patches or API changes occuring upstream among sub-dependencies.  In addition to the Develop-Release Cycle Workflow, LamAna endorses both "hands-off" and "hands-on" options for installing.

### Hands-off Approach: unpinned

By default, LamAna (and most Pythonic packages) assume that dependencies are coded with minimal API changes that intentionally break code.  With this in mind, users can simply:

```bash
$ pip install lamana
```

This command searches for dependencies in the `install_requires` key of the `setup.py` file.  Dependencies are intentionally "unpinned" here, which means a user will (usually) download the latest version of every dependency listed.

### Hands-on Approach: pinned

One issue with unpinned dependencies is updates can break a package.  You can protect against such unpredictable changes by pinning dependencies.  In such a case that a dependency fails to install, the user rely on a few **pinned** alternatives:

#### 1. Install from `conda` (recommended)

At times, dependencies fail to install correctly through `pip`.  This method proves be the most reliable for installing dependencies:

```bash
$ conda install numpy pandas matplotlib  
$ pip install lamana  
```

#### 2. Install from a `wheel` file (optional)

Sometimes installing from source is slow.  You can force the latter installation method to install with faster binaries. 

```bash
$ pip install lamana --use-wheel            # binary  
```

#### 3. Install with a `requirements.txt` File (optional)

We can recreate the original dependency environment in which the release was originally developed and working.  The recreated environment installs pinned dependencies from a frozen `requirements.txt` file.  

Acquire the file by downloading from the release version on GitHub website.  Locating this file is not hard.  Each release is shipped with this a requirements.txt file.  The file simply needs to be download from the archives of the correct version of lamana hosted at [GitHub releases](https://github.com/par2/lamana/releases) or search on [PyPI](https://pypi.python.org/pypi). Extract the file to your computer and run the commands. 


```bash
$ pip install -r </path/to/requirements.txt>    
$ pip install lamana                        # source            
```

Instructions for setting up consistent virtural environments are described in the [testing section](testing.ipynb#Regression-Tests-with-nb_conda_kernels).

## Installing dependencies from source (intermediate)

**In the absence of Anaconda**, installing dependendencies from source can be tedious and arduous, specifically `numpy`, `pandas` and `matplotlib`.  Here are some reasons and tips [1](https://stackoverflow.com/questions/26473681/pip-install-numpy-throws-an-error-ascii-codec-cant-decode-byte-0xe2), [2](https://stackoverflow.com/questions/25674612/ubuntu-14-04-pip-cannot-upgrade-matplotllib) for installing dependencies if they are not setup on your system.

On Debian-based systems, install the following pre-requisites.  

    $ apt-get install build-essential python3-dev


On Windows systems, be certain to install the [appropriate Visual Studio C-compilers](https://matthew-brett.github.io/pydagogue/python_msvc.html).

This option tends to be painful and are discoruaged if possible.

### Dependencies

- Mandatory Dependencies
    - [numpy](http://www.numpy.org/) 
    - [matplotlib](http://matplotlib.org/)
    - [pandas](http://pandas.pydata.org/)
    - [openpyxl](https://openpyxl.readthedocs.io/en/default/)
    - [xlrd](http://xlrd.readthedocs.io/en/latest/)

- Recommended Dependencies
    - [notebook](http://jupyter.org/)


The following table shows a chart of tested builds compatible with LamAna:
    
|lamana|python|dependency|OS|
|:--|:--|:--|:--|
|0.4.8|2.7.6, 2.7.10, 3.3, 3.4, 3.5, 3.5.1| numpy==1.10.1, pandas==0.16.2, matplotlib==1.5.0|linux, local win(?)|
|0.4.9|2.7, 3.3, 3.4, 3.5, 3.5.1| conda==3.19.0, numpy==1.10.1, pandas==0.16.2, matplotlib==1.4.3|linux, win(?)|
|0.4.10|2.7, 2.7.11, 3.3, 3.4, 3.5, 3.5.1| conda==3.19.0, numpy==1.10.2, pandas==0.17.1, matplotlib==1.5.1|linux |
|0.4.10|2.7 (x32, x64), 3.4 (x32), 3.5 (x32, x64)| conda==3.19.0, numpy==1.10.2, pandas==0.17.1, matplotlib==1.5.1| win |
|latest|2.7 (x32, x64), 3.4 (x32), 3.5 (x32, x64)| conda>=3.19.0, numpy>=1.10.2, pandas>=0.17.1, matplotlib>=1.5.1| win |